# Cleaning the data

In [1]:
import pandas as pd
df = pd.read_csv('my_strava_activities.csv', index_col=0)

### Choosing which data to use

One of the first decisions we must make is about which data we plan to use. In general we should select only those values or variables for which we have a sufficent number of entries. Let's look once again at the type of activities

In [2]:
df['type'].value_counts()

Run            585
Ride           346
Swim            30
VirtualRide     15
Name: type, dtype: int64

For our analysis we will choose to focuse only on activities which are runs or rides.

In [3]:
df = df[df['type'].isin(['Ride','Run'])]

### Dealing with missing values

Our data has some missing values

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931 entries, 0 to 975
Data columns (total 9 columns):
average_speed           931 non-null float64
average_heartrate       410 non-null float64
average_watts           337 non-null float64
distance                931 non-null float64
elapsed_time            931 non-null object
total_elevation_gain    931 non-null float64
type                    931 non-null object
commute                 931 non-null bool
start_date_local        931 non-null object
dtypes: bool(1), float64(5), object(3)
memory usage: 66.4+ KB


Since `average_watts` is a cycling specific measure let's fill in the values for runs as `0`. 

In [5]:
df.loc[df['type']=='Run', 'average_watts'] = 0

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931 entries, 0 to 975
Data columns (total 9 columns):
average_speed           931 non-null float64
average_heartrate       410 non-null float64
average_watts           922 non-null float64
distance                931 non-null float64
elapsed_time            931 non-null object
total_elevation_gain    931 non-null float64
type                    931 non-null object
commute                 931 non-null bool
start_date_local        931 non-null object
dtypes: bool(1), float64(5), object(3)
memory usage: 66.4+ KB


For the remainder missing values, we can drop the observations corresponding to them

In [7]:
df.drop('average_heartrate', axis=1, inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 922 entries, 0 to 975
Data columns (total 8 columns):
average_speed           922 non-null float64
average_watts           922 non-null float64
distance                922 non-null float64
elapsed_time            922 non-null object
total_elevation_gain    922 non-null float64
type                    922 non-null object
commute                 922 non-null bool
start_date_local        922 non-null object
dtypes: bool(1), float64(4), object(3)
memory usage: 58.5+ KB


### Transforming distance to kilometers

The given distance values are in meters. Let's transform them to kilometers and round to 2 decimal digits

In [8]:
df['distance'] = df['distance']/1000
df['distance'] = df['distance'].round(2)
df['distance'].head()

0     51.69
1    133.13
2    108.42
3     59.02
4     23.83
Name: distance, dtype: float64

### Transform average speed to km/h

The given values are in meters/second. Let's transform them to to kilometers/hour and round to 2 decimal places. 

In [9]:
df['average_speed'] = df['average_speed'] * 18/5
df['average_speed']  = df['average_speed'].round(2)
df['average_speed'].head()

0    19.67
1    25.01
2    24.67
3    24.32
4    26.83
Name: average_speed, dtype: float64

### Transform elapsed time to minutes

This seems a bit more complicated since the values are strings. Let's start by getting an idea of the type of values we have

In [10]:
df['elapsed_time'].str.len().value_counts()

7     917
8       4
16      1
Name: elapsed_time, dtype: int64

Let's see the format for each of these lengths

In [11]:
df[df['elapsed_time'].str.len() == 7]['elapsed_time'].head()

0    3:23:50
1    6:17:54
2    4:39:40
3    2:38:43
4    0:55:01
Name: elapsed_time, dtype: object

In [12]:
df[df['elapsed_time'].str.len() == 8]['elapsed_time']

33     18:00:55
256    10:07:16
279    10:01:59
471    10:35:32
Name: elapsed_time, dtype: object

In [13]:
df[df['elapsed_time'].str.len() == 16]['elapsed_time']

583    3 days, 23:05:23
Name: elapsed_time, dtype: object

It seems that we have three types of possible formats:
* h:mm:ss
* hh:mm:ss
* d days, hh:mm:ss

Let's try to convert all of these to total number of minutes. We will write a function that given a string:

1. Check if the string is of the type `d days, hh:mm:ss`

2. If yes, extract the integer `d` and return `hh:mm:ss`, else pass.

3. Extract the integers `hh`, `mm` and `ss`

4. Calculate the total number of minutes

In [14]:
def get_minutes(time_str):
    if ',' in time_str:
        d = time_str[0]
        time_str = time_str.split(',')[1]
    else:
        d = 0
    h, m, s = time_str.split(':') 
    total_min = (int(d)*24+int(h))*60+int(m)+int(s)/60
    return int(total_min)

## Aside: the methods `apply()` and `map()`

So far we have seen that we can perform transformation on columns of a DataFrame by taking advantage of vectorized operations. However, sometimes the transformations that we want to perform are more complicated than vetorized operations allow. 

These scenarios usually fall under one of the following two cases:

* we have function and we want to apply this function to each individual entry of the column 

* we have a dictionary that maps current values of the column to new values

Luckily, the pandas Series object has specific methods for each of these two cases. If we have a function we use the `Series.apply()` method

<img src="pandas.Series.apply.png" alt="drawing" style="width: 700px;"/>

And if we have a dictionary we use the `Series.map()` method

<img src="pandas.Series.map.png" alt="drawing" style="width: 700px;"/>

We are in the first case here so we can write

In [15]:
df['time_min'] = df['elapsed_time'].apply(get_minutes)

Let's get rid of the original column

In [16]:
df.drop('elapsed_time', axis=1, inplace=True)
df.head()

,average_speed,average_watts,distance,total_elevation_gain,type,commute,start_date_local,time_min
0,19.67,131.7,51.69,1414.0,Ride,False,2018-05-21T09:59:24,203
1,25.01,127.8,133.13,2280.0,Ride,False,2018-05-20T08:46:44,377
2,24.67,136.4,108.42,1950.0,Ride,False,2018-05-19T08:12:35,279
3,24.32,135.3,59.02,937.0,Ride,False,2018-05-17T17:50:55,158
4,26.83,117.7,23.83,122.0,Ride,False,2018-05-17T07:15:06,55


## Dealing with time data

Pandas was first invented for analyzing financial time series, so understandably it has quite the machinery built in to deal manipulating dates, aggreggrating over different time periods and much more. 

Pandas has a single object to encapsulate date and time called a `Timestamp`.

<img src="pandas.Timestamp.png" alt="drawing" style="width: 700px;"/>

The constructor is extremely flexible and handles a wide range of inputs. Let's see if we can pass the strings from our column `start_date_local` directly to the `pandas.Timestamp()` constructor

In [17]:
df['start_date_local'][0]

'2018-05-21T09:59:24'

In [18]:
pd.Timestamp(df['start_date_local'][0])

Timestamp('2018-05-21 09:59:24')

However if we try to pass the entire column to the constructor we will get an error. Instead we can use the `pandas.to_datetime()` function that is capable of converting an entire Series into `Timestamp` objects

<img src="pandas.to_datetime.png" alt="drawing" style="width: 700px;"/>

Let's try to convert our column `start_date_local`

In [19]:
#convert start date to datetime 
df['start_date_local'] = pd.to_datetime(df['start_date_local'])

In [20]:
df['start_date_local'][0]

Timestamp('2018-05-21 09:59:24')

Let's now set this column as our new index

In [21]:
df.set_index('start_date_local', inplace=True)
df.head()

,average_speed,average_watts,distance,total_elevation_gain,type,commute,time_min
start_date_local,,,,,,,
2018-05-21 09:59:24,19.67,131.7,51.69,1414.0,Ride,False,203
2018-05-20 08:46:44,25.01,127.8,133.13,2280.0,Ride,False,377
2018-05-19 08:12:35,24.67,136.4,108.42,1950.0,Ride,False,279
2018-05-17 17:50:55,24.32,135.3,59.02,937.0,Ride,False,158
2018-05-17 07:15:06,26.83,117.7,23.83,122.0,Ride,False,55


In [22]:
df.index

DatetimeIndex(['2018-05-21 09:59:24', '2018-05-20 08:46:44',
               '2018-05-19 08:12:35', '2018-05-17 17:50:55',
               '2018-05-17 07:15:06', '2018-05-16 18:09:43',
               '2018-05-15 19:49:01', '2018-05-14 19:38:03',
               '2018-05-12 09:13:34', '2018-05-05 09:47:42',
               ...
               '2013-10-05 09:36:57', '2013-09-28 16:34:35',
               '2013-08-04 20:15:43', '2013-07-12 20:46:31',
               '2013-07-12 20:31:32', '2013-07-05 18:55:07',
               '2013-06-23 12:43:01', '2013-06-16 08:59:56',
               '2013-06-14 13:40:09', '2013-06-01 09:02:40'],
              dtype='datetime64[ns]', name='start_date_local', length=922, freq=None)

The nice thing about using the `Timestamp` objects is that they have a large number attributes and methods. For example

In [23]:
df.index.year

Int64Index([2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018,
            ...
            2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013],
           dtype='int64', name='start_date_local', length=922)

In [24]:
df.index.month

Int64Index([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
            ...
            10,  9,  8,  7,  7,  7,  6,  6,  6,  6],
           dtype='int64', name='start_date_local', length=922)

In [25]:
df.index.dayofweek

Int64Index([0, 6, 5, 3, 3, 2, 1, 0, 5, 5,
            ...
            5, 5, 6, 4, 4, 4, 6, 6, 4, 5],
           dtype='int64', name='start_date_local', length=922)

We can also round `Timestamps` to our desired level of accuracy

In [26]:
df.index = df.index.round('D', inplace=True)
df.index.rename('date', inplace=True)

In [27]:
df.head()

,average_speed,average_watts,distance,total_elevation_gain,type,commute,time_min
date,,,,,,,
2018-05-21,19.67,131.7,51.69,1414.0,Ride,False,203
2018-05-20,25.01,127.8,133.13,2280.0,Ride,False,377
2018-05-19,24.67,136.4,108.42,1950.0,Ride,False,279
2018-05-18,24.32,135.3,59.02,937.0,Ride,False,158
2018-05-17,26.83,117.7,23.83,122.0,Ride,False,55


## Aside: the advantages of a DatetimeIndex

One of the advantages of changing our index to the `date` column is that now we can take advantage of more flexible slicing options. For example if we wanted to bring up the activities from this last month we can simply type

In [28]:
df.loc['May 2018']

,average_speed,average_watts,distance,total_elevation_gain,type,commute,time_min
date,,,,,,,
2018-05-21,19.67,131.7,51.69,1414.0,Ride,False,203
2018-05-20,25.01,127.8,133.13,2280.0,Ride,False,377
2018-05-19,24.67,136.4,108.42,1950.0,Ride,False,279
2018-05-18,24.32,135.3,59.02,937.0,Ride,False,158
2018-05-17,26.83,117.7,23.83,122.0,Ride,False,55
2018-05-17,22.53,129.0,23.77,504.0,Ride,False,66
2018-05-16,10.31,0.0,10.85,347.3,Run,False,68
2018-05-15,10.54,0.0,5.00,50.0,Run,False,31
2018-05-12,26.24,121.9,129.49,1833.0,Ride,False,355


# Outliers

Now that we cleaned all of our individual colums the next time is to remove any outliers from our data. The definition of what constitudes and outlier is depends on the particular data set and applications.

For our example we will say that a point is an outlier if it is more than the standard deviation away from the mean. We will then remove all observations which contain an outlier in any of the column.

However for this to make sense we must define the mean and standard with respect to the type of activity. In order to do this we need to learn about a major taks in data analysis: splitting data into groups


<img src="pandas.DataFrame.groupby.png" alt="drawing" style="width: 700px;"/>

In [29]:
grouped = df.drop('commute', axis=1).groupby('type')
grouped

Let's investigate a bit this groupby object

In [30]:
grouped.describe()

average_speed                                                          \
             count       mean       std    min    25%    50%    75%    max   
type                                                                         
Ride         337.0  24.334362  3.138615  13.58  22.30  24.32  26.25  36.86   
Run          585.0  10.188615  1.305560   4.04   9.55  10.30  10.97  14.50   

     average_watts               ...   time_min         total_elevation_gain  \
             count        mean   ...        75%     max                count   
type                             ...                                           
Ride         337.0  133.594362   ...      288.0  5705.0                337.0   
Run          585.0    0.000000   ...      105.0   458.0                585.0   

                                                                   
             mean         std   min    25%    50%     75%     max  
type                                                               
Ride  1128.331751  975.227155  33.0  366.3  884.6  1530.4  5799.2  
Run    167.535556  304.856658   0.0   24.2   75.2   160.4  2253.4  

[2 rows x 40 columns]

We can get a list of all the group names

Most commonly we will use the DataFrameGroupBy object to perform aggregations or transformations.

In [31]:
grouped.mean()

,average_speed,average_watts,distance,total_elevation_gain,time_min
type,,,,,
Ride,24.334362,133.594362,70.086499,1128.331751,226.412463
Run,10.188615,0.000000,10.563402,167.535556,75.030769


In [32]:
grouped.max()

,average_speed,average_watts,distance,total_elevation_gain,time_min
type,,,,,
Ride,36.86,269.3,210.09,5799.2,5705
Run,14.50,0.0,50.66,2253.4,458


### Applying functions on to a groupby object

The groupby object has four methods that accept a functionto perform a calculation on each group:


* `agg`: returns a scalar,  is called multiple times per group (once for each non-grouping column)
* `filter`: returns a boolean 
* `transform`: returns a Series with the same length as the passed group, is called multiple times per group (once for each non-grouping column)
* `apply`: flexible return type, is called only once per group

In our case we want to apply a function to each group that removes the outliers. Since this is a per column operation we must use the `tranform` method.

What would our function look like?

In [33]:
def f(group):
    return (group-group.mean()).abs().div(group.std()) 

In [34]:
grouped.transform(f).head()

,average_speed,average_watts,distance,total_elevation_gain,time_min
date,,,,,
2018-05-21,1.486121,0.065934,0.445696,0.292925,0.070490
2018-05-20,0.215266,0.201675,1.527370,1.180923,0.453386
2018-05-19,0.106938,0.097651,0.928715,0.842540,0.158329
2018-05-18,0.004576,0.059365,0.268111,0.196192,0.205975
2018-05-17,0.795140,0.553210,1.120667,1.031895,0.516085


In [35]:
outliers = grouped.transform(f)>2.5

In [36]:
df[outliers.any(axis=1)].shape

(50, 7)

In [37]:
df = df[~outliers.any(axis=1)]
df.shape

(872, 7)

In [38]:
df.to_csv('activities_clean.csv')

# Exercise: 

1. In this exercise we group the activities according to the following three groups:

    * runs
    * commute run
    * non-commute rides
    * commute rides

2. Calculate the total time of all activities per group as a percentage of the total time  across all activities.

3. Compute the hardness per group where hardness is defined as 

```
hardness = 0.2*average_speed.mean() + 
+ 0.5*distance.mean() 
+ 0.3*total_elevation_gain.mean()
```

Are commutes or non commutes harder?

In [39]:
grouped = df[['time_min', 'type', 'commute']].groupby(['type','commute'])

In [40]:
result = grouped.sum()
result

time_min
type commute          
Ride False       60641
     True         1122
Run  False       36870
     True          777

In [43]:
total_time = df['time_min'].sum()
result['percentage'] = result['time_min']/total_time*100
result

time_min  percentage
type commute                      
Ride False       60641   61.000905
     True         1122    1.128659
Run  False       36870   37.088824
     True          777    0.781612

In [44]:
def hardness(group):
    return 0.2*group['average_speed'].mean()+0.5*group['distance'].mean()+0.3*group['total_elevation_gain'].mean()

In [45]:
df.groupby(['type', 'commute']).apply(hardness)

type  commute
Ride  False      361.812855
      True        68.534700
Run   False       42.978413
      True        39.009889
dtype: float64